<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [1]:

#!pip install tensorflow==1.14.0 keras==2.2.4
import numpy as np
from tensorflow.keras.utils import normalize
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

Using TensorFlow backend.


((404, 13), (404,), (102, 13), (102,))

In [2]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)

In [3]:
# Buid Model

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

np.random.seed(812)

boston_housing_model = Sequential()
#Input
boston_housing_model.add(Dense(16, input_dim=13, activation='relu'))

# Hidden
boston_housing_model.add(Dense(16, activation='relu'))
# Hidden
boston_housing_model.add(Dense(16, activation='relu'))
# Output
boston_housing_model.add(Dense(1, activation='relu'))

#Compile
boston_housing_model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mse'])





boston_housing_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 785
Trainable params: 785
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Hyper Parameters
batch_size = 32
epochs = 1000

# Fit Model

history = boston_housing_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1, verbose=0)
scores = boston_housing_model.evaluate(x_test, y_test)
print(f'{boston_housing_model.metrics_names[1]}: {scores[1]}')

102/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [5]:
import matplotlib.pyplot as plt

fig, ((ax1,ax2)) = plt.subplots(1,2)
ax1.plot(history.history['loss'], color = 'r')
ax1.set_title("Loss")
ax2.plot(history.history['mse'], color = 'g')
ax2.set_title("MSE");

# Linear Regression

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [7]:
linear_model = LinearRegression()

In [8]:
# Fitting the model
linear_model.fit(x_train, y_train)

/Users/maximevacher-materno/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [9]:
# Predict the model

y_pred = linear_model.predict(x_test)

In [10]:
# Mean squared error MSE

mse = mean_squared_error(y_test, y_pred)
print(f'MSE:\n', mse)

MSE:
 18.165510493496477


## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [11]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from keras.datasets import fashion_mnist 
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [12]:
# Hyper Parameters

batch_size = 64
num_classes = 10
epochs = 20

In [13]:
X_train = x_train.reshape(60000, 784)
X_test = x_test.reshape(10000, 784)

In [14]:
# X Variable Types
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [15]:
y_train[2] 

0

In [16]:
# Correct Encoding on Y
# What softmax expects = [0,0,0,0,0,1,0,0,0,0]

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [17]:
fashion_mnist_model = Sequential()

# Input => Hidden
fashion_mnist_model.add(Dense(16, input_dim=784, activation='relu'))
# Hidden
fashion_mnist_model.add(Dense(16, activation='relu'))
# Hidden
fashion_mnist_model.add(Dense(16, activation='relu'))
# Hidden
fashion_mnist_model.add(Dense(16, activation='relu'))
# Output
fashion_mnist_model.add(Dense(10,activation='softmax'))

#Compile
fashion_mnist_model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

fashion_mnist_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                12560     
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 10)                170       
Total params: 13,546
Trainable params: 13,546
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = fashion_mnist_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1, verbose=1)
scores = fashion_mnist_model.evaluate(X_test, y_test)
print(f'{fashion_mnist_model.metrics_names[1]}: {scores[1]*100}')

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 2s 32us/sample - loss: 1.7232 - accuracy: 0.4329 - val_loss: 1.1271 - val_accuracy: 0.5793
Epoch 2/20
54000/54000 [==============================] - 1s 20us/sample - loss: 0.8842 - accuracy: 0.6377 - val_loss: 0.7820 - val_accuracy: 0.6682
Epoch 3/20
54000/54000 [==============================] - 1s 20us/sample - loss: 0.7099 - accuracy: 0.7334 - val_loss: 0.7032 - val_accuracy: 0.7158
Epoch 4/20
54000/54000 [==============================] - 1s 21us/sample - loss: 0.6522 - accuracy: 0.7553 - val_loss: 0.6402 - val_accuracy: 0.7755
Epoch 5/20
54000/54000 [==============================] - 1s 19us/sample - loss: 0.6272 - accuracy: 0.7632 - val_loss: 0.6340 - val_accuracy: 0.7592
Epoch 6/20
54000/54000 [==============================] - 1s 19us/sample - loss: 0.6158 - accuracy: 0.7673 - val_loss: 0.6572 - val_accuracy: 0.7457
Epoch 7/20
54000/54000 [==============================] -

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.